In [20]:
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage,HumanMessage
from typing import Any, TypedDict,Annotated, Literal
from pydantic import BaseModel,Field
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:

class TweetEvaluation(BaseModel):
    evaluation: Literal["approved", "needs_improvement"] = Field(..., description="Final evaluation result.")
    feedback: str = Field(..., description="feedback for the tweet.")



In [22]:
generator = ChatGoogleGenerativeAI(model="gemini-2.5-flash") # Specify the model here
evaluator = ChatGoogleGenerativeAI(model="gemini-2.5-flash") # Specify the model here
optimizer = ChatGoogleGenerativeAI(model="gemini-2.5-flash") # Specify the model here


In [23]:
structured_evaluator_llm = evaluator.with_structured_output(TweetEvaluation)

In [ ]:
class TweetState(TypedDict):
    topic: str
    tweet: str
    evaluation: Literal["approved", "needs_improvement"]
    feedback: str
    iteration: int
    max_iterations: int
    tweet_history: list[str]
    feedback_history: list[str]

    

In [ ]:
def generate_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english
""")
    ]

    response = generator.invoke(messages).content

    return {
        'tweet': response,
        'tweet_history': state.get("tweet_history", []) + [response]
    }

# %% Evaluate tweet
def evaluate_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
        HumanMessage(content=f"""
Evaluate the following tweet:

Tweet: "{state['tweet']}"

Use the criteria below to evaluate the tweet:

1. Originality – Is this fresh, or have you seen it a hundred times before?  
2. Humor – Did it genuinely make you smile, laugh, or chuckle?  
3. Punchiness – Is it short, sharp, and scroll-stopping?  
4. Virality Potential – Would people retweet or share it?  
5. Format – Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Don’t end with generic, throwaway, or deflating lines

### Respond ONLY in structured format:
- evaluation: "approved" or "needs_improvement"  
- feedback: One paragraph explaining the strengths and weaknesses 
""")
    ]

    response = structured_evaluator_llm.invoke(messages)

    return {
        'evaluation': response.evaluation,
        'feedback': response.feedback,
        'feedback_history': state.get("feedback_history", []) + [response.feedback]
    }

# %% Optimize tweet
def optimize_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
""")
    ]

    response = optimizer.invoke(messages).content
    iteration = state['iteration'] + 1

    return {
        'tweet': response,
        'iteration': iteration,
        'tweet_history': state.get("tweet_history", []) + [response]
    }

# %% Routing
def route_evaluation(state: TweetState):
    if state['evaluation'] == "approved" or state['iteration'] >= state['max_iterations']:
        return END
    else:
        return 'optimize_tweet'
    

In [32]:
graph = StateGraph(TweetState)

graph.add_node('generate_tweet', generate_tweet)
graph.add_node('evaluate_tweet', evaluate_tweet)
graph.add_node('optimize_tweet', optimize_tweet)

graph.add_edge(START, 'generate_tweet')
graph.add_edge('generate_tweet', 'evaluate_tweet')

graph.add_conditional_edges('evaluate_tweet', route_evaluation)


graph.add_edge('optimize_tweet', 'evaluate_tweet')

workflow = graph.compile()

# %% Run
initial_state = {
    "topic": "Hogwarts professors on Twitter",
    "iteration": 1,
    "max_iterations": 5,
    "tweet_history": [],
    "feedback_history": []
}

result = workflow.invoke(initial_state)
print(result)

{'topic': 'Hogwarts professors on Twitter', 'tweet': 'Hogwarts professors on Twitter: Flitwick trying to go viral with a "Wingardium Leviosa" TikTok dance tutorial (too many emojis). Meanwhile, Snape\'s burner account is just subtweeting Gryffindors and rating potion ingredients. #HogwartsOnX', 'evaluation': 'approved', 'feedback': 'This tweet demonstrates excellent originality and humor by perfectly capturing the essence of Flitwick and Snape in a modern social media context. The specific scenarios, like Flitwick\'s "Wingardium Leviosa" TikTok and Snape\'s burner account subtweeting Gryffindors, are both fresh and genuinely funny, making it highly relatable and shareable. The tweet is punchy, well-formatted, and under the character limit, ensuring strong virality potential. The use of the hashtag #HogwartsOnX is a clever touch that enhances its appeal.', 'iteration': 1, 'max_iterations': 5}
